In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [60]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [61]:
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          transform=transform)

Files already downloaded and verified


In [62]:
print(cifar_train)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [63]:
print(cifar_test)

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [64]:
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=32, shuffle=True)

# 代码练习部分，使用pytorch构建网络

In [65]:
class LeNet(nn.Module):
    # 一般在__init__中定义网络需要的操作算子，比如卷积、全连接算子等等
    def __init__(self):
        super(LeNet, self).__init__()
        #此处进行代码练习
        self.conv1=nn.Conv2d(3,10,3,padding=1,stride=1)
        self.conv2=nn.Conv2d(10,20,3,padding=1,stride=1)
        self.fc1 = nn.Linear(20*16*16,500)
        self.fc2 = nn.Linear(500,10)
    # 定义了前向传播的运算，只需要像写普通的python算数运算那样就可以了
    def forward(self, x):
        #进行代码练习
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out    
        

In [66]:
# 如果你没有GPU，那么可以忽略device相关的代码
#device = torch.device("cuda:0")
#net = LeNet().to(device)

In [67]:
import torch.optim as optim

# CrossEntropyLoss就是我们需要的损失函数
net = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [68]:
print("Start Training...")
for epoch in range(5):
    # 用一个变量来记录每100个batch的平均loss
    loss100 = 0.0
    # 使用dataloader操作数据
    for i, data in enumerate(trainloader):
        inputs, labels = data
  #      inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss100 += loss.item()
        if i % 100 == 99:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss100 / 100))
            loss100 = 0.0

print("Done Training!")

Start Training...
[Epoch 1, Batch   100] loss: 2.298
[Epoch 1, Batch   200] loss: 2.277
[Epoch 1, Batch   300] loss: 2.224
[Epoch 1, Batch   400] loss: 2.130
[Epoch 1, Batch   500] loss: 2.019
[Epoch 1, Batch   600] loss: 1.953
[Epoch 1, Batch   700] loss: 1.923
[Epoch 1, Batch   800] loss: 1.899
[Epoch 1, Batch   900] loss: 1.837
[Epoch 1, Batch  1000] loss: 1.801
[Epoch 1, Batch  1100] loss: 1.779
[Epoch 1, Batch  1200] loss: 1.762
[Epoch 1, Batch  1300] loss: 1.744
[Epoch 1, Batch  1400] loss: 1.702
[Epoch 1, Batch  1500] loss: 1.685
[Epoch 2, Batch   100] loss: 1.644
[Epoch 2, Batch   200] loss: 1.619
[Epoch 2, Batch   300] loss: 1.572
[Epoch 2, Batch   400] loss: 1.565
[Epoch 2, Batch   500] loss: 1.504
[Epoch 2, Batch   600] loss: 1.503
[Epoch 2, Batch   700] loss: 1.515
[Epoch 2, Batch   800] loss: 1.480
[Epoch 2, Batch   900] loss: 1.454
[Epoch 2, Batch  1000] loss: 1.462
[Epoch 2, Batch  1100] loss: 1.438
[Epoch 2, Batch  1200] loss: 1.432
[Epoch 2, Batch  1300] loss: 1.412
[E

In [69]:
# 首先要把梯度清零，不然PyTorch每次计算梯度会累加，不清零的话第二次算的梯度等于第一次加第二次的       
optimizer.zero_grad()
# 计算前向传播的输出
outputs = net(inputs)
# 根据输出计算loss
loss = criterion(outputs, labels)
# 算完loss之后进行反向梯度传播，这个过程之后梯度会记录在变量中
loss.backward()
# 用计算的梯度去做优化
optimizer.step()

In [70]:
# 构造测试的dataloader
dataiter = iter(testloader)
# 预测正确的数量和总数量
correct = 0
total = 0
# 使用torch.no_grad的话在前向传播中不记录梯度，节省内存
with torch.no_grad():
    for data in testloader:
        images, labels = data
        #images, labels = images.to(device), labels.to(device)
        # 预测
        outputs = net(images)
        # 我们的网络输出的实际上是个概率分布，去最大概率的哪一项作为预测分类
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 60 %
